In [1]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df1 = pd.read_csv('dataset_v1').set_index('building_id')
df2 = pd.read_csv('dataset_v2').drop(columns=['Unnamed: 0']).set_index('building_id')

removed_features = [
    'roof_type',
    'land_surface_condition',
    'foundation_type',
    'ground_floor_type',
    'other_floor_type',
    'position',
    'plan_configuration',
    'legal_ownership_status',
    'damage_grade'
]

encoded_features = [
    'roof_type_C0',
    'roof_type_C1',
    'land_surface_condition_C0',
    'land_surface_condition_C1',
    'foundation_type_C0',
    'foundation_type_C1',
    'foundation_type_C2',
    'ground_floor_type_C0',
    'ground_floor_type_C1',
    'ground_floor_type_C2',
    'other_floor_type_C0',
    'other_floor_type_C1',
    'other_floor_type_C2',
    'position_C0',
    'position_C1',
    'position_C2',
    'plan_configuration_C0',
    'plan_configuration_C1',
    'plan_configuration_C2',
    'plan_configuration_C3',
    'legal_ownership_status_C0',
    'legal_ownership_status_C1',
    'legal_ownership_status_C2'
]

for col in encoded_features:
    df1[col] = df2[col]
    
df1 = df1.drop(columns=removed_features)

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 53 columns):
 #   Column                                  Non-Null Count   Dtype
---  ------                                  --------------   -----
 0   geo_level_1_id                          260601 non-null  int64
 1   geo_level_2_id                          260601 non-null  int64
 2   geo_level_3_id                          260601 non-null  int64
 3   count_floors_pre_eq                     260601 non-null  int64
 4   age                                     260601 non-null  int64
 5   area_percentage                         260601 non-null  int64
 6   height_percentage                       260601 non-null  int64
 7   has_superstructure_adobe_mud            260601 non-null  bool 
 8   has_superstructure_mud_mortar_stone     260601 non-null  bool 
 9   has_superstructure_stone_flag           260601 non-null  bool 
 10  has_superstructure_cement_mortar_stone  260601 non-null  bool 


In [4]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler

# the model
from catboost import Pool, CatBoostClassifier

# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

x = df1
y = pd.read_csv('train_labels.csv').set_index('building_id')['damage_grade']

In [5]:
catboost_pipe = make_pipeline(
    StandardScaler(),
    CatBoostClassifier(iterations=100, random_state=2018,loss_function='MultiClass')
)

param_grid = {
    'catboostclassifier__learning_rate': np.linspace(0.1, 1.0, 10),
    'catboostclassifier__depth': list(range(2, 13))
}

gs = GridSearchCV(catboost_pipe, param_grid, cv=5, verbose=3)

In [6]:
gs.fit(x, y)

Fitting 5 folds for each of 110 candidates, totalling 550 fits
0:	learn: 1.0591081	total: 69ms	remaining: 6.83s
1:	learn: 1.0268196	total: 91.1ms	remaining: 4.47s
2:	learn: 0.9989176	total: 112ms	remaining: 3.62s
3:	learn: 0.9745372	total: 135ms	remaining: 3.24s
4:	learn: 0.9539142	total: 155ms	remaining: 2.95s
5:	learn: 0.9360312	total: 176ms	remaining: 2.75s
6:	learn: 0.9207184	total: 195ms	remaining: 2.59s
7:	learn: 0.9083496	total: 217ms	remaining: 2.5s
8:	learn: 0.8961427	total: 238ms	remaining: 2.41s
9:	learn: 0.8856002	total: 258ms	remaining: 2.32s
10:	learn: 0.8768996	total: 281ms	remaining: 2.28s
11:	learn: 0.8674671	total: 305ms	remaining: 2.23s
12:	learn: 0.8601372	total: 326ms	remaining: 2.18s
13:	learn: 0.8529850	total: 348ms	remaining: 2.14s
14:	learn: 0.8473442	total: 369ms	remaining: 2.09s
15:	learn: 0.8426544	total: 390ms	remaining: 2.05s
16:	learn: 0.8377396	total: 412ms	remaining: 2.01s
17:	learn: 0.8301902	total: 435ms	remaining: 1.98s
18:	learn: 0.8262333	total: 45

59:	learn: 0.7568576	total: 1.31s	remaining: 875ms
60:	learn: 0.7563139	total: 1.33s	remaining: 852ms
61:	learn: 0.7555091	total: 1.35s	remaining: 830ms
62:	learn: 0.7547179	total: 1.37s	remaining: 807ms
63:	learn: 0.7540574	total: 1.4s	remaining: 786ms
64:	learn: 0.7537640	total: 1.42s	remaining: 766ms
65:	learn: 0.7534451	total: 1.44s	remaining: 744ms
66:	learn: 0.7528569	total: 1.47s	remaining: 722ms
67:	learn: 0.7507376	total: 1.49s	remaining: 702ms
68:	learn: 0.7502893	total: 1.51s	remaining: 681ms
69:	learn: 0.7494033	total: 1.54s	remaining: 660ms
70:	learn: 0.7491227	total: 1.56s	remaining: 639ms
71:	learn: 0.7478302	total: 1.59s	remaining: 618ms
72:	learn: 0.7473659	total: 1.61s	remaining: 596ms
73:	learn: 0.7468975	total: 1.64s	remaining: 575ms
74:	learn: 0.7466648	total: 1.66s	remaining: 552ms
75:	learn: 0.7462977	total: 1.68s	remaining: 531ms
76:	learn: 0.7459210	total: 1.7s	remaining: 508ms
77:	learn: 0.7455936	total: 1.72s	remaining: 486ms
78:	learn: 0.7451379	total: 1.75s

21:	learn: 0.8171271	total: 469ms	remaining: 1.66s
22:	learn: 0.8146078	total: 491ms	remaining: 1.64s
23:	learn: 0.8091052	total: 514ms	remaining: 1.63s
24:	learn: 0.8057896	total: 536ms	remaining: 1.61s
25:	learn: 0.8033907	total: 557ms	remaining: 1.59s
26:	learn: 0.8015095	total: 577ms	remaining: 1.56s
27:	learn: 0.7975800	total: 598ms	remaining: 1.54s
28:	learn: 0.7953966	total: 618ms	remaining: 1.51s
29:	learn: 0.7926000	total: 639ms	remaining: 1.49s
30:	learn: 0.7901713	total: 662ms	remaining: 1.47s
31:	learn: 0.7889277	total: 683ms	remaining: 1.45s
32:	learn: 0.7873234	total: 708ms	remaining: 1.44s
33:	learn: 0.7851997	total: 729ms	remaining: 1.42s
34:	learn: 0.7841311	total: 749ms	remaining: 1.39s
35:	learn: 0.7825691	total: 773ms	remaining: 1.37s
36:	learn: 0.7814462	total: 792ms	remaining: 1.35s
37:	learn: 0.7800567	total: 812ms	remaining: 1.32s
38:	learn: 0.7783644	total: 831ms	remaining: 1.3s
39:	learn: 0.7756858	total: 855ms	remaining: 1.28s
40:	learn: 0.7745739	total: 874m

82:	learn: 0.7436855	total: 1.75s	remaining: 359ms
83:	learn: 0.7434262	total: 1.77s	remaining: 338ms
84:	learn: 0.7430961	total: 1.79s	remaining: 316ms
85:	learn: 0.7427317	total: 1.81s	remaining: 295ms
86:	learn: 0.7417242	total: 1.84s	remaining: 274ms
87:	learn: 0.7414192	total: 1.86s	remaining: 253ms
88:	learn: 0.7411800	total: 1.88s	remaining: 232ms
89:	learn: 0.7407766	total: 1.9s	remaining: 211ms
90:	learn: 0.7405735	total: 1.92s	remaining: 190ms
91:	learn: 0.7400923	total: 1.94s	remaining: 169ms
92:	learn: 0.7398655	total: 1.97s	remaining: 148ms
93:	learn: 0.7395243	total: 1.99s	remaining: 127ms
94:	learn: 0.7380307	total: 2.01s	remaining: 106ms
95:	learn: 0.7377436	total: 2.03s	remaining: 84.6ms
96:	learn: 0.7375892	total: 2.05s	remaining: 63.5ms
97:	learn: 0.7373865	total: 2.07s	remaining: 42.3ms
98:	learn: 0.7372548	total: 2.09s	remaining: 21.1ms
99:	learn: 0.7367954	total: 2.12s	remaining: 0us
[CV 5/5] END catboostclassifier__depth=2, catboostclassifier__learning_rate=0.1;,

39:	learn: 0.7413779	total: 874ms	remaining: 1.31s
40:	learn: 0.7407679	total: 898ms	remaining: 1.29s
41:	learn: 0.7399947	total: 916ms	remaining: 1.26s
42:	learn: 0.7394977	total: 936ms	remaining: 1.24s
43:	learn: 0.7388102	total: 959ms	remaining: 1.22s
44:	learn: 0.7383442	total: 982ms	remaining: 1.2s
45:	learn: 0.7377236	total: 1s	remaining: 1.18s
46:	learn: 0.7372148	total: 1.02s	remaining: 1.16s
47:	learn: 0.7367318	total: 1.05s	remaining: 1.13s
48:	learn: 0.7342049	total: 1.07s	remaining: 1.11s
49:	learn: 0.7324605	total: 1.1s	remaining: 1.1s
50:	learn: 0.7315852	total: 1.12s	remaining: 1.07s
51:	learn: 0.7310810	total: 1.14s	remaining: 1.05s
52:	learn: 0.7306874	total: 1.16s	remaining: 1.03s
53:	learn: 0.7302421	total: 1.18s	remaining: 1.01s
54:	learn: 0.7297431	total: 1.21s	remaining: 987ms
55:	learn: 0.7293472	total: 1.22s	remaining: 962ms
56:	learn: 0.7289714	total: 1.25s	remaining: 940ms
57:	learn: 0.7283803	total: 1.27s	remaining: 918ms
58:	learn: 0.7279012	total: 1.29s	rem

...

83:	learn: 0.4462127	total: 36.3s	remaining: 6.92s
84:	learn: 0.4452669	total: 36.8s	remaining: 6.49s
85:	learn: 0.4432076	total: 37.2s	remaining: 6.06s
86:	learn: 0.4406566	total: 37.6s	remaining: 5.62s
87:	learn: 0.4388249	total: 38.1s	remaining: 5.19s
88:	learn: 0.4375785	total: 38.5s	remaining: 4.76s
89:	learn: 0.4362334	total: 38.9s	remaining: 4.33s
90:	learn: 0.4347575	total: 39.4s	remaining: 3.89s
91:	learn: 0.4328711	total: 39.8s	remaining: 3.46s
92:	learn: 0.4312137	total: 40.2s	remaining: 3.03s
93:	learn: 0.4298025	total: 40.7s	remaining: 2.6s
94:	learn: 0.4287890	total: 41.1s	remaining: 2.16s
95:	learn: 0.4273965	total: 41.5s	remaining: 1.73s
96:	learn: 0.4265386	total: 42s	remaining: 1.3s
97:	learn: 0.4253741	total: 42.4s	remaining: 865ms
98:	learn: 0.4237767	total: 42.8s	remaining: 433ms
99:	learn: 0.4227321	total: 43.3s	remaining: 0us
[CV 5/5] END catboostclassifier__depth=12, catboostclassifier__learning_rate=0.9;, score=0.711 total time=  44.6s
0:	learn: 0.7723292	total

41:	learn: 0.5257474	total: 18.2s	remaining: 25.1s
42:	learn: 0.5233367	total: 18.6s	remaining: 24.7s
43:	learn: 0.5199333	total: 19.1s	remaining: 24.3s
44:	learn: 0.5183103	total: 19.5s	remaining: 23.8s
45:	learn: 0.5162715	total: 19.9s	remaining: 23.4s
46:	learn: 0.5131654	total: 20.4s	remaining: 22.9s
47:	learn: 0.5100551	total: 20.8s	remaining: 22.5s
48:	learn: 0.5084802	total: 21.2s	remaining: 22.1s
49:	learn: 0.5058136	total: 21.7s	remaining: 21.7s
50:	learn: 0.5045943	total: 22.1s	remaining: 21.2s
51:	learn: 0.5021919	total: 22.5s	remaining: 20.8s
52:	learn: 0.4992752	total: 23s	remaining: 20.4s
53:	learn: 0.4974156	total: 23.4s	remaining: 19.9s
54:	learn: 0.4946955	total: 23.8s	remaining: 19.5s
55:	learn: 0.4934051	total: 24.3s	remaining: 19.1s
56:	learn: 0.4904210	total: 24.7s	remaining: 18.6s
57:	learn: 0.4874477	total: 25.1s	remaining: 18.2s
58:	learn: 0.4849746	total: 25.6s	remaining: 17.8s
59:	learn: 0.4817875	total: 26s	remaining: 17.3s
60:	learn: 0.4796042	total: 26.4s	r

0:	learn: 0.7707059	total: 426ms	remaining: 42.1s
1:	learn: 0.7186742	total: 855ms	remaining: 41.9s
2:	learn: 0.6956371	total: 1.29s	remaining: 41.8s
3:	learn: 0.6824497	total: 1.74s	remaining: 41.7s
4:	learn: 0.6736252	total: 2.17s	remaining: 41.3s
5:	learn: 0.6663402	total: 2.6s	remaining: 40.8s
6:	learn: 0.6578922	total: 3.04s	remaining: 40.4s
7:	learn: 0.6514066	total: 3.47s	remaining: 39.9s
8:	learn: 0.6484075	total: 3.9s	remaining: 39.4s
9:	learn: 0.6424078	total: 4.33s	remaining: 39s
10:	learn: 0.6359422	total: 4.76s	remaining: 38.5s
11:	learn: 0.6300935	total: 5.17s	remaining: 37.9s
12:	learn: 0.6256990	total: 5.59s	remaining: 37.4s
13:	learn: 0.6210606	total: 6.03s	remaining: 37s
14:	learn: 0.6159355	total: 6.45s	remaining: 36.6s
15:	learn: 0.6093155	total: 6.89s	remaining: 36.2s
16:	learn: 0.6057006	total: 7.31s	remaining: 35.7s
17:	learn: 0.6028430	total: 7.74s	remaining: 35.3s
18:	learn: 0.5997220	total: 8.17s	remaining: 34.8s
19:	learn: 0.5954008	total: 8.61s	remaining: 34

61:	learn: 0.4770476	total: 26.7s	remaining: 16.4s
62:	learn: 0.4740860	total: 27.2s	remaining: 16s
63:	learn: 0.4720332	total: 27.6s	remaining: 15.5s
64:	learn: 0.4704654	total: 28s	remaining: 15.1s
65:	learn: 0.4685888	total: 28.5s	remaining: 14.7s
66:	learn: 0.4659100	total: 28.9s	remaining: 14.2s
67:	learn: 0.4623783	total: 29.3s	remaining: 13.8s
68:	learn: 0.4604086	total: 29.8s	remaining: 13.4s
69:	learn: 0.4582687	total: 30.2s	remaining: 12.9s
70:	learn: 0.4555789	total: 30.6s	remaining: 12.5s
71:	learn: 0.4541376	total: 31.1s	remaining: 12.1s
72:	learn: 0.4523925	total: 31.5s	remaining: 11.7s
73:	learn: 0.4510075	total: 32s	remaining: 11.2s
74:	learn: 0.4495454	total: 32.4s	remaining: 10.8s
75:	learn: 0.4466541	total: 32.8s	remaining: 10.4s
76:	learn: 0.4447486	total: 33.3s	remaining: 9.93s
77:	learn: 0.4429419	total: 33.7s	remaining: 9.5s
78:	learn: 0.4411672	total: 34.1s	remaining: 9.07s
79:	learn: 0.4394625	total: 34.5s	remaining: 8.64s
80:	learn: 0.4383401	total: 35s	remain

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('catboostclassifier',
                                        <catboost.core.CatBoostClassifier object at 0x7f8155c3abe0>)]),
             param_grid={'catboostclassifier__depth': [2, 3, 4, 5, 6, 7, 8, 9,
                                                       10, 11, 12],
                         'catboostclassifier__learning_rate': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             verbose=3)

In [11]:
gs.best_params_

{'catboostclassifier__depth': 11, 'catboostclassifier__learning_rate': 0.4}

In [18]:
test_values = pd.read_csv('encoded_test_values.csv').set_index('building_id')

removed_features = [
    'roof_type',
    'land_surface_condition',
    'foundation_type',
    'ground_floor_type',
    'other_floor_type',
    'position',
    'plan_configuration',
    'legal_ownership_status'
]

test_values = test_values.drop(columns=removed_features)

for col in test_values.columns:
    if col.startswith('has_superstructure') or col.startswith('has_secondary_use'):
        test_values[col] = test_values[col].astype('bool')


In [20]:
test_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86868 entries, 300051 to 501372
Data columns (total 53 columns):
 #   Column                                  Non-Null Count  Dtype
---  ------                                  --------------  -----
 0   geo_level_1_id                          86868 non-null  int64
 1   geo_level_2_id                          86868 non-null  int64
 2   geo_level_3_id                          86868 non-null  int64
 3   count_floors_pre_eq                     86868 non-null  int64
 4   age                                     86868 non-null  int64
 5   area_percentage                         86868 non-null  int64
 6   height_percentage                       86868 non-null  int64
 7   has_superstructure_adobe_mud            86868 non-null  bool 
 8   has_superstructure_mud_mortar_stone     86868 non-null  bool 
 9   has_superstructure_stone_flag           86868 non-null  bool 
 10  has_superstructure_cement_mortar_stone  86868 non-null  bool 
 11  has_super

In [21]:
prediction = gs.predict(test_values)

In [23]:
sub_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [24]:
submission = pd.DataFrame(data=prediction, columns=sub_format.columns, index=sub_format.index)
submission.to_csv('submission_catboost_pipeline.csv')

In [25]:
submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,2
421793,3
